In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import optuna
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Add this at the start of your Kaggle notebook
# !pip install -U scikit-learn==1.2.2
# !pip install -U imbalanced-learn==0.10.1

In [ ]:
import matplotlib.pyplot as plt
import missingno as mso
import seaborn as sns
import warnings
import scipy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import ttest_ind
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

In [ ]:
data_path = '/kaggle/input/chocolate-bar-ratings/flavors_of_cacao.csv'
cc = pd.read_csv(data_path)
cc

In [ ]:
import re
for column in cc.columns:
    cc.columns = cc.columns.str.replace(r'\s*\n\s*', ' ', regex=True)
cc.info()

In [ ]:
cc.rename(columns={
    'Company (Maker-if known)': 'Company',
    'Specific Bean Origin or Bar Name': 'BarName',
    'Review Date':'ReviewDate',
    'Cocoa Percent':'CocoaPercent',
    'Company Location':'CompanyLocation',
    'Bean Type':'BeanType',
    'Broad Bean Origin' : 'BroadBeanOrigin',
}, inplace=True)

In [ ]:
cc.info()

In [ ]:
cc.isnull().sum()

In [ ]:
cc['CocoaPercent'] = cc['CocoaPercent'].str.replace('%', '').astype(float)
cc['CocoaPercent'].info()

In [ ]:
cc['BeanType'] = (cc['BeanType']
                 .str.strip()  # Menghapus leading/trailing spaces
                 .replace('', np.nan))  # Mengganti string kosong dengan NaN
cc['BeanType'] = cc['BeanType'].replace(np.nan, cc['BeanType'].mode()[0])

In [ ]:
cc['ReviewDate'] = pd.to_datetime(cc['ReviewDate'].astype(str), format='%Y', errors='coerce')
cc['ReviewDate'] = cc['ReviewDate'].dt.year

In [ ]:
cc.isnull().sum()
cc['BroadBeanOrigin'] = cc['BroadBeanOrigin'].fillna(cc['BroadBeanOrigin'].mode()[0])

In [ ]:
cc_numerik = cc.select_dtypes(include=np.number)
cc_numerik.info()
cc_object = cc.select_dtypes(include='object')
cc_object.info()

# cc = pd.get_dummies(cc, columns=['Company','BarName','CompanyLocation','BeanType'], drop_first=True)

In [ ]:
# le = LabelEncoder()
# categorical_features = ['BeanType', 'BroadBeanOrigin', 'CompanyLocation']
# for columns in categorical_features:
#     cc[columns] = le.fit_transform(cc[columns])

label_encoders = {}
categorical_features = ['BeanType', 'BroadBeanOrigin', 'CompanyLocation']

for column in categorical_features:  
    label_encoders[column] = LabelEncoder()
    cc[column] = label_encoders[column].fit_transform(cc[column])
    
company_freq = cc['Company'].value_counts(normalize=True)
cc['Company'] = cc['Company'].map(company_freq)

In [ ]:
cc.info()

# Modeling

In [ ]:
X = cc.drop(columns='BroadBeanOrigin')
y = cc['BroadBeanOrigin']

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# 1. Check class distribution
print("Original class distribution:")
print(y.value_counts())

# 2. Filter classes with too few samples
min_samples = 6
value_counts = y.value_counts()
valid_classes = value_counts[value_counts >= min_samples].index
mask = y.isin(valid_classes)
X_filtered = X[mask]
y_filtered = y[mask]

print("\nClass distribution after filtering:")
print(y_filtered.value_counts())

# 3. Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_filtered, y_filtered)

# 4. Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, 
    y_resampled, 
    test_size=0.2, 
    random_state=42
)

print("\nFinal class distribution:")
print(pd.Series(y_resampled).value_counts())

In [ ]:
# from imblearn.over_sampling import SMOTE


# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X, y)
# X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
LRclassifier = LogisticRegression(solver='saga', random_state=42)
LRclassifier.fit(X_train, y_train)

In [ ]:
y_pred = LRclassifier.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

from sklearn.metrics import accuracy_score
LRAcc = accuracy_score(y_pred,y_test)
print('LR accuracy: {:.2f}%'.format(LRAcc*100))

In [ ]:
def f1_score(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)

    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    return f1

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# def objective(trial):
#     n_estimators = trial.suggest_int('n_estimators', 10, 200)
#     max_depth = trial.suggest_int('max_depth', 3, 20)
#     min_samples_split= trial.suggest_int('min_samples_split', 2, 20)
    
#     model = RandomForestClassifier(
#         n_estimators=n_estimators,
#         max_depth=max_depth,
#         random_state=42,
#         bootstrap=True,
#         min_samples_split=min_samples_split,
#         max_features='sqrt'
#     )
    
#     model.fit(X_train, y_train)

#     y_pred = model.predict(X_test)

#     score = f1_score(y_test, y_pred)

#     return score

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials = 20)

# best_params = study.best_params
# print("Best Parameter:", best_params)

# best_model = RandomForestClassifier(
#     n_estimators = best_params['n_estimators'],
#     max_depth = best_params['max_depth'], 
#     random_state=42,
#     bootstrap=True,
#     min_samples_split=5,
#     max_features='sqrt',    
# )

# best_model.fit(X_train, y_train)

# y_pred = best_model.predict(X_test)

# final_f1 = f1_score(y_test,y_pred)
# print("Final f1:", final_f1)